In [10]:
# from qiskit import QuantumCircuit, Aer, transpile, execute
# from qiskit.circuit import Parameter
# import numpy as np
# import matplotlib.pyplot as plt

# theta = Parameter('θ')
# qc = QuantumCircuit(1, 1)  # Add 1 classical bit for measurement

# # Apply gates as per the specification
# qc.h(0)
# # qc.p(0.5*theta, 0)
# qc.rz(-1.0 * theta, 0)
# qc.h(0)
# qc.measure(0, 0)

# # Backend setup
# backend = Aer.get_backend('qasm_simulator')

# # Values of theta
# # theta_vals = np.linspace(0, 4 * np.pi, 1000)
# theta_vals = np.arange(1e-3, 3*np.pi + 1e-3, 1e-2)
# probs = []

# # Get probabilities for |1⟩ for various theta
# for val in theta_vals:
#     qc_temp = qc.bind_parameters({theta: val})
#     counts = execute(qc_temp, backend, shots=100000).result().get_counts()
#     probs.append(counts.get('1', 0) / 100000)

# # Differentiate probabilities to get dp/dθ
# dp_dtheta = np.gradient(probs, theta_vals)

# # Classical Fisher Information is square of the gradient
# cfi = dp_dtheta**2

# # Plot CFI vs theta
# plt.plot(theta_vals, cfi)
# plt.title('Classical Fisher Information vs. θ')
# plt.xlabel('θ')
# plt.ylabel('CFI')
# plt.show()


In [11]:
import numpy as np

from qiskit import Aer, QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.aer import AerSimulator

def get_probabilities(circuit, params):
    sim = AerSimulator()
    circ = transpile(circuit, sim)
    result = sim.run(circ.bind_parameters(params)).result()
    counts = result.get_counts()
    shots = sum(counts.values())
    probabilities = {k: v / shots for k, v in counts.items()}
    return probabilities

def compute_cfim(circuit, theta, delta=1e-5):
    N = len(theta)
    CFIM = np.zeros((N, N))
    base_probs = get_probabilities(circuit, theta)

    for i in range(N):
        theta_plus = theta.copy()
        theta_plus[i] += delta
        probs_plus = get_probabilities(circuit, theta_plus)
        dprobs_i = {k: (probs_plus.get(k, 0) - base_probs.get(k, 0)) / delta for k in base_probs.keys()}

        for j in range(N):
            theta_plus = theta.copy()
            theta_plus[j] += delta
            probs_plus = get_probabilities(circuit, theta_plus)
            dprobs_j = {k: (probs_plus.get(k, 0) - base_probs.get(k, 0)) / delta for k in base_probs.keys()}

            for k, base_prob in base_probs.items():
                CFIM[i, j] += dprobs_i[k] * dprobs_j[k] / base_prob

    return CFIM

# Example Usage:
theta = [Parameter('θ0'), Parameter('θ1')]
circuit = QuantumCircuit(2)
circuit.rx(theta[0], 0)
circuit.ry(theta[1], 1)
circuit.measure_all()

theta_values = [np.pi/4, np.pi/6]
cfim = compute_cfim(circuit, theta_values)
print(cfim)


ModuleNotFoundError: No module named 'qiskit.aer'